In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from matplotlib import pyplot as plt
from scipy import stats as st
import math

In [2]:
# Attempt to import data, handle exception, give feedback if successful
file_path = "../vehicles_us.csv"

try:
    vehicles_df = pd.read_csv(file_path)
except FileNotFoundError as error_msg:
    print(f"Error reading file: {error_msg}. Try again!")
else:
    print(f"The file at path: [{file_path}] was imported.")
    print("The import was saved to the variable: [vehicles_df]")

The file at path: [../vehicles_us.csv] was imported.
The import was saved to the variable: [vehicles_df]


## 1. Data Cleaning 

In [3]:
# Print general information
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


**Observations**

The dataset contains 51,525 entries. Among these, various columns exhibit missing data, namely:

- `model_year`
- `cylinders`
- `odometer`
- `paint_color`
- `is_4wd`

For the analysis, a standardized approach will be employed to address these missing values:

- Missing values in `model_year`, `cylinders`, and `odometer` will be replaced with their respective statistical means.
- The `paint_color` column's missing values will be filled with the categorical value `"Unknown"`.
- Missing values in the `is_4wd` column will be filled with `0`. From an analytical perspective, it's preferable to inaccurately classify cars without four-wheel drive as not having it than vice versa.

Regarding duplicated values, there's no need for concern since all columns could potentially contain duplicates. For instance, multiple cars might share the same mileage or posting date. Similarly, there could be duplicates of the same cars.

Furthermore, certain columns will be repurposed and derived from existing ones:
1. Convert the `date_posted` column to `datetime` format.
2. Create `year_posted`, `month_posted`, and `day_posted` columns from `date_posted`, enabling comparisons at daily, monthly, and yearly intervals.
3. Introduce an `id` column to assign each car a unique identifier. Currently, the index serves as the ID, but adding a unique ID ensures persistence even if the index is reset.
4. Split the existing `model` column into `make` and `model` columns. The `make` column will denote the brand (e.g., BMW, Honda, Subaru), while the `model` column will specify the model (e.g., X5, F-150, Sonata). This division facilitates comparisons by brand or specific models.

In [4]:
# Updating numerical 
try:
    vehicles_df["model_year"]=vehicles_df["model_year"].fillna(round(vehicles_df["model_year"].mean()))
    vehicles_df["cylinders"] = vehicles_df["cylinders"].fillna(round(vehicles_df["cylinders"].mean()))
    vehicles_df["odometer"] = vehicles_df["odometer"].fillna(round(vehicles_df["odometer"].mean()))
    vehicles_df['paint_color'] = vehicles_df['paint_color'].fillna("unknown")
    vehicles_df['is_4wd'] = vehicles_df['is_4wd'].fillna(0)
except:
    print("There was an error! Try again.")
else:
    print(f"Missing values for 'model_year': {vehicles_df['model_year'].isna().sum()}")
    print(f"Missing values for 'cylinders': {vehicles_df['cylinders'].isna().sum()}")
    print(f"Missing values for 'odometer': {vehicles_df['odometer'].isna().sum()}")
    print(f"Missing values for 'paint_color': {vehicles_df['paint_color'].isna().sum()}")
    print(f"Missing values for 'is_4wd': {vehicles_df['is_4wd'].isna().sum()}")

Missing values for 'model_year': 0
Missing values for 'cylinders': 0
Missing values for 'odometer': 0
Missing values for 'paint_color': 0
Missing values for 'is_4wd': 0


In [5]:
# Change the data type of the entire column
columns_to_convert = ['model_year', 'cylinders', 'odometer', 'is_4wd']
vehicles_df[columns_to_convert] = vehicles_df[columns_to_convert].astype(int)

In [6]:
# Feature engineering dates based on the date_posted column
vehicles_df['date_posted'] = pd.to_datetime(vehicles_df['date_posted'])
vehicles_df['year_posted'] = vehicles_df['date_posted'].dt.year
vehicles_df['month_posted'] = vehicles_df['date_posted'].dt.month
vehicles_df['day_posted'] = vehicles_df['date_posted'].dt.day

# Separate out the make and model so analysis can be done on both
vehicles_df[['make', 'model']] = vehicles_df['model'].str.split(' ', n=1, expand=True)
vehicles_df.insert(2, 'make', vehicles_df.pop('make'))
vehicles_df.insert(3, 'model', vehicles_df.pop('model'))

In [7]:
# Create an ID column based on index. If the index is reset later, the ID will still map back to original
vehicles_df.insert(0, 'id', vehicles_df.index)

In [8]:
# Look for any values that should be combined
unique_models = vehicles_df['model'].unique()
print(unique_models)

['x5' 'f-150' 'sonata' '200' '300' 'camry' 'pilot' 'sorento'
 'silverado 1500' 'accord' '1500' 'yukon' 'cherokee' 'traverse' 'elantra'
 'tahoe' 'rav4' 'silverado' 'wrangler' 'malibu' 'fusion se' 'impala'
 'corvette' 'liberty' 'camry le' 'altima' 'outback' 'highlander' 'charger'
 'tacoma' 'equinox' 'rogue' 'benze sprinter 2500' 'cr-v' 'grand cherokee'
 '4runner' 'focus' 'civic' 'soul' 'colorado' 'f150 supercrew cab xlt'
 'camaro lt coupe 2d' 'cruze' 'mustang' 'silverado 3500hd'
 'frontier crew cab sv' 'impreza' 'grand cherokee laredo' 'versa'
 'f-250 sd' 'silverado 1500 crew' 'f250 super duty' 'camaro'
 'mustang gt coupe 2d' 'forester' 'explorer' 'f-350 sd' 'edge' 'maxima'
 'f-250' 'sentra' 'f150' 'suburban' 'expedition' 'grand caravan' 'taurus'
 'tl' 'f350 super duty' 'ranger' 'sierra' 'santa fe' 'escape'
 'sierra 2500hd' 'civic lx' 'sierra 1500' 'odyssey' 'escalade' 'jetta'
 'corolla' 'town & country' 'passat' 'prius' 'enclave' 'fusion' 'tundra'
 '3500' '2500' 'frontier' 'silverado 25

In [9]:
# combining 'unqiue' values
vehicles_df.loc[vehicles_df['model'] == "silverado", 'model'] = 'silverado 1500'
vehicles_df.loc[vehicles_df['model'] == "f150", 'model'] = 'f-150'

In [30]:
vehicles_df

,id,price,model_year,make,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,year_posted,month_posted,day_posted
0,0,9400,2011,bmw,x5,good,6,gas,145000,automatic,SUV,unknown,1,2018-06-23,19,2018,6,23
1,1,25500,2010,ford,f-150,good,6,gas,88705,automatic,pickup,white,1,2018-10-19,50,2018,10,19
2,2,5500,2013,hyundai,sonata,like new,4,gas,110000,automatic,sedan,red,0,2019-02-07,79,2019,2,7
3,3,1500,2003,ford,f-150,fair,8,gas,115553,automatic,pickup,unknown,0,2019-03-22,9,2019,3,22
4,4,14900,2017,chrysler,200,excellent,4,gas,80903,automatic,sedan,black,0,2019-04-02,28,2019,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,51520,9249,2013,nissan,maxima,like new,6,gas,88136,automatic,sedan,black,0,2018-10-03,37,2018,10,3
51521,51521,2700,2002,honda,civic,salvage,4,gas,181500,automatic,sedan,white,0,2018-11-14,22,2018,11,14
51522,51522,3950,2009,hyundai,sonata,excellent,4,gas,128000,automatic,sedan,blue,0,2018-11-15,32,2018,11,15
51523,51523,7455,2013,toyota,corolla,good,4,gas,139573,automatic,sedan,black,0,2018-07-02,71,2018,7,2


## 2. Data Exploration

In [28]:
# price distribution

price_hist_fig = px.histogram(vehicles_df, x="price", nbins=50)
price_hist_fig.update_layout(title_text="Distribution of Vehicle Price", yaxis_title='Count', xaxis_title='Price', bargap=0.2)
price_hist_fig.update_traces(marker_color='rgb(136, 204, 238)')
price_hist_fig.show()

In [12]:
# model_year distribution

model_year_hist_fig = px.histogram(vehicles_df, x="model_year",  nbins=200)
model_year_hist_fig.update_layout(title_text="Distribution of Vehicle Year", yaxis_title='Count', xaxis_title='Year of Model',  bargap=0.2)
model_year_hist_fig.show()

In [29]:
# make distribution

make_dist_fig = px.histogram(vehicles_df, x='make')
make_dist_fig.update_xaxes(tickangle=45) 
make_dist_fig.update_layout(title_text="Distribution of Vehicle Make", yaxis_title='Count', xaxis_title='Vehicle Make', bargap=0.2)
make_dist_fig.update_traces(marker_color='rgb(248, 156, 116)')
make_dist_fig.show()

In [26]:
# model distribution (top 20)

model_counts = vehicles_df['model'].value_counts()
top_n_models = model_counts.head(25)
model_dist_fig = px.histogram(x=top_n_models.index, y=top_n_models.values)
model_dist_fig.update_xaxes(tickangle=45) 
model_dist_fig.update_layout(title_text="Distribution of Vehicle Model (Top 25)", yaxis_title='Count', xaxis_title='Vehicle Model', bargap=0.2)
model_dist_fig.update_traces(marker_color='rgb(102, 194, 165)')
model_dist_fig.show()

In [18]:
# make distribution by condition

make_cond_dist_fig = px.histogram(vehicles_df, x='make', color='condition')
make_cond_dist_fig.update_layout(title_text="Distribution of Vehicle Make by Condition", yaxis_title='Count', xaxis_title='Vehicle Make', height=800)
make_cond_dist_fig.show()